# 🚀 Kaggle-Ready GPT-2 Training Notebook

**This notebook automatically downloads all code from GitHub!**

Just upload this `.ipynb` file to Kaggle and run all cells.

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 1: SETUP - AUTO-CLONE FROM GITHUB
# ═══════════════════════════════════════════════════════════
import os
import sys

# Check if we're on Kaggle or Colab
is_kaggle = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
is_colab = 'COLAB_GPU' in os.environ

if is_kaggle or is_colab:
    print("🔍 Running on cloud platform - cloning repository...")
    
    # Clone the repository
    !git clone https://github.com/shaunthecomputerscientist/LLM_FROM_SCRATCH_IMPLEMENTATION.git
    
    # Change to repo directory
    os.chdir('LLM_FROM_SCRATCH_IMPLEMENTATION')
    
    # Add to Python path
    sys.path.insert(0, os.getcwd())
    
    # Install requirements
    !pip install -q -r requirements.txt
    
    print("✅ Repository cloned and ready!")
    print(f"📂 Current directory: {os.getcwd()}")
    
else:
    print("💻 Running locally - using local files")
    from pathlib import Path
    sys.path.insert(0, str(Path.cwd()))

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 2: IMPORTS
# ═══════════════════════════════════════════════════════════
import torch
import tiktoken
import matplotlib.pyplot as plt

from llm_from_scratch.GPT2Model.gpt2 import GPTModel
from llm_from_scratch.Dataset.loader import create_dataloader_v1
from llm_from_scratch.Trainer.trainer import (
    train_model_simple,
    generate_text_simple
)

print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 3: CONFIGURATION
# ═══════════════════════════════════════════════════════════
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cfg = {
    # Model architecture (smaller for Kaggle)
    "vocab_size": 50257,
    "context_length": 256,     # Reduced for speed
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 6,             # Reduced from 12
    "drop_rate": 0.1,
    "qkv_bias": False,
    "use_flash": torch.cuda.is_available(),  # Auto-detect
    
    # Dataloader
    "batch_size": 4,
    "stride": 64,
    "train_ratio": 0.90,
}

print("📋 Configuration:")
for k, v in cfg.items():
    print(f"   {k}: {v}")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 4: LOAD DATASET
# ═══════════════════════════════════════════════════════════
from datasets import load_dataset

# Download small sample from HuggingFace
print("📥 Downloading dataset...")
dataset = load_dataset(
    "HuggingFaceFW/fineweb-edu",
    name="sample-10BT",
    split="train",
    streaming=True
)

# Take first 50 documents (~5MB text)
raw_text = ""
for i, doc in enumerate(dataset):
    if i >= 50:
        break
    raw_text += doc['text'] + "\n\n"

print(f"✅ Dataset: {len(raw_text):,} characters")

# Split and create loaders
split_idx = int(cfg["train_ratio"] * len(raw_text))
train_data = raw_text[:split_idx]
val_data = raw_text[split_idx:]

train_loader = create_dataloader_v1(
    train_data,
    batch_size=cfg["batch_size"],
    max_length=cfg["context_length"],
    stride=cfg["stride"]
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=cfg["batch_size"],
    max_length=cfg["context_length"],
    stride=cfg["stride"]
)

print(f"📊 Train batches: {len(train_loader)}")
print(f"📊 Val batches: {len(val_loader)}")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 5: INITIALIZE MODEL
# ═══════════════════════════════════════════════════════════
model = GPTModel(cfg).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)
tokenizer = tiktoken.get_encoding("gpt2")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"🔢 Total parameters: {total_params:,}")
print(f"💾 Model size: {total_params * 4 / 1024**2:.1f} MB (FP32)")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 6: TRAIN MODEL
# ═══════════════════════════════════════════════════════════
print("🚂 Starting training...\n")

train_losses, val_losses, tokens_seen = train_model_simple(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device=device,
    num_epochs=5,
    eval_freq=10,
    eval_iter=5,
    start_context="Every effort moves",
    tokenizer=tokenizer,
    memory_efficient=torch.cuda.is_available(),
    accumulation_steps=4
)

print("\n✅ Training complete!")

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 7: PLOT TRAINING CURVES
# ═══════════════════════════════════════════════════════════
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', color='blue')
plt.plot(val_losses, label='Val Loss', color='orange')
plt.xlabel('Evaluation Step')
plt.ylabel('Loss')
plt.title('Training Progress')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(tokens_seen, train_losses, color='blue')
plt.xlabel('Tokens Seen')
plt.ylabel('Loss')
plt.title('Loss vs Tokens')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 8: GENERATE TEXT
# ═══════════════════════════════════════════════════════════
model.eval()

prompts = [
    "Every effort moves",
    "The artist pondered",
    "In the beginning"
]

print("🎨 Generated Text:\n" + "="*60)

for prompt in prompts:
    encoded = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)
    
    output = generate_text_simple(
        model=model,
        idx=encoded,
        max_new_tokens=30,
        context_size=cfg["context_length"],
        temperature=0.7,
        top_k=10
    )
    
    generated = tokenizer.decode(output.squeeze(0).tolist())
    print(f"\n📝 {generated}")
    print("-" * 60)

In [ ]:
# ═══════════════════════════════════════════════════════════
# CELL 9: SAVE MODEL
# ═══════════════════════════════════════════════════════════
# Save checkpoint
checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'config': cfg,
    'train_losses': train_losses,
    'val_losses': val_losses,
    'tokens_seen': tokens_seen
}

torch.save(checkpoint, 'gpt2_checkpoint.pt')
print("✅ Full checkpoint saved: gpt2_checkpoint.pt")

# Save just weights (smaller file)
torch.save(model.state_dict(), 'gpt2_weights.pt')
print("✅ Weights saved: gpt2_weights.pt")

print("\n📥 Download these files from Kaggle output folder!")